In [109]:
import json
import asyncio
import aiohttp
import glob
import requests
import pandas as pd
import warnings
import time
import numpy as np
from decimal import Decimal
from pandas.api.types import is_bool_dtype, is_numeric_dtype, is_object_dtype, is_datetime64_dtype, is_float_dtype, is_int64_dtype

In [2]:
with open('../secrets.json') as f:
    access_token = json.load(f)['AIRTABLE_ACCESS_TOKEN']

In [3]:
headers = {"Authorization" : f"Bearer {access_token}",
           "Content-Type"  : "application/json"}

In [4]:
api_url = "https://api.airtable.com"

In [5]:
app_id = "appBgReq44W6yAMqS"

In [6]:
def table_exists(table_name):
    url = f"{api_url}/v0/{app_id}/{table_name}"
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return True
    elif response.status_code == 403:
        return False
    else:
        response.raise_for_status()

In [7]:
root_path = "../datasets/rank_1"

In [8]:
table_names = []
table_data = []

for file in glob.glob(f"{root_path}/*.csv"):
    name = file.split('/')[-1].split('.')[0]
    table_names.append(name)
    df = pd.read_csv(file)
    table_data.append(df)

In [9]:
# POST
create_table_endpoint = f"{api_url}/v0/meta/bases/{app_id}/tables"
def create_table(name, fields):
    payload = {
        "name": name,
        "description": "",
        "fields": fields
    }
    response = requests.post(create_table_endpoint, headers=headers, data=json.dumps(payload))
    return response.json()

In [10]:
def is_date_col(df, col):
    date_pattern = r'^(19|20)\d\d[- /.](0[1-9]|1[012])[- /.](0[1-9]|[12][0-9]|3[01])$'
        
    try:
        is_date = df[col].str.match(date_pattern, na=True) 
        true_count = is_date.sum()
        total_count = is_date.notna().sum()
        
        if total_count > 0:
            percentage_true = true_count / total_count
            if percentage_true > 0.99:
                return True
    except:
        return False

In [44]:
def is_bool_col(df, col):
    nuniques = len(pd.unique(df[col]))
    return nuniques == 2 and df[col].dtype == 'int64'

In [62]:
def get_field_type(df, col):
    dtype = df[col].dtype
    
    if is_bool_dtype(dtype):
        return 'checkbox'
    elif is_datetime64_dtype(dtype):
        return 'date'
    elif is_numeric_dtype(dtype):
        return 'number'
    else:
        return 'singleLineText'

In [69]:
def df_to_fields(df):
    field_types = [get_field_type(df, col) for col in df.columns]
    fields = [{"name": col, "type": field_types[i]} for i, col in enumerate(df.columns)]
    
    for i, field in enumerate(fields):
        if field['type'] == 'date':
            fields[i]['options'] = {
                "dateFormat": {
                    "name": "iso"
                }
            }
            
        elif field['type'] == 'number':
            col = df[df.columns[i]]
            precision = 0 if col.dtype == 'int64' else 6
            fields[i]['options'] = {
                "precision": precision
            }

        elif field['type'] == 'checkbox':
            fields[i]['options'] = {
                "color": "greenBright",
                "icon": "check"
            }
            
    return fields

In [104]:
fields_data = []
formatted_data = []

for i, df in enumerate(table_data):

    # Process the columns for field type inference
    for col in df.columns:
        
        # Convert dates to datetime
        if is_date_col(df, col):
            with warnings.catch_warnings():
                warnings.simplefilter("ignore")
                df[col] = pd.to_datetime(df[col], errors='coerce')

        # Convert 1/0 to True/False
        if is_bool_col(df, col):
            df[col] = df[col].replace({0: False, 1: True})

    # Infer field types
    fields = df_to_fields(df)

    # Format data for passing into API
    df = df.fillna('')
    
    for col in df.columns:
        if is_datetime64_dtype(df[col]):
            # Format to ISO string
            df[col] = df[col].dt.strftime('%Y-%m-%d')
            
    fields_data.append(fields)
    formatted_data.append(df)

In [72]:
for name, fields in zip(table_names, fields_data):
    if table_exists(name):
        continue
    create_table(name, fields)

In [76]:
def prepare_data(df):
    df = df.where(pd.notnull(df), None)
    records = []
    for _, row in df.iterrows():
        fields = {col: row[col] for col in df.columns}
        records.append({"fields": fields})
    return records

In [141]:
prepared_data = []

for df in formatted_data:
    prepared_df = prepare_data(df)
    prepared_data.append(prepared_df)

In [81]:
def upload_data(data, table_name):
    
    table_endpoint = f"{api_url}/v0/{app_id}/{table_name}"
    
    for i in range(0, len(data), 10):
        batch = data[i:i + 10]
        payload = {"records": batch}
        
        response = requests.post(table_endpoint, headers=headers, json=payload)
        
        if response.status_code == 200:
            print(f"Batch {i // 10 + 1} uploaded successfully.")
        else:
            print(f"Error in batch {i // 10 + 1}: {response.json()}")

In [94]:
uploaded_states = ["False"]*len(table_names)

In [148]:
i =-1
for table_name, data in zip(table_names, prepared_data):
    i += 1

    if uploaded_states[i] == True:
        print(f"Table {table_name} data already uploaded")
        continue
    
    if not table_exists(table_name):
        print(f"Could not find table {table_name}")
        continue

    #result = await post_data(table_name, data, retries=2, delay=3)
    result = upload_data(data, table_name)
    print(result)
    uploaded_states[i] = True
    
    res = input("continue?")
    if res != 'y':
        break

Table binary_classifications data already uploaded
Table canciones_aleatorias_con_fechas_y_datos data already uploaded
Table charts data already uploaded
Table complete_filled_top_1_songs data already uploaded
Table features data already uploaded
Table features_scaled data already uploaded
Table lyric_sentiment data already uploaded
Table lyric_sentiment_scaled data already uploaded
Table query data already uploaded
Table track_info data already uploaded
Table unique data already uploaded
